# Project configuration file

Since this file will also be uploaded, this avoids exposing sensitive data by configuration global paths. So define a project.ini file with a `images_folder` variable that will point towards an images folder called `fashion`.

In [ ]:
import configparser
import pathlib

In [ ]:
config_file = configparser.ConfigParser()
config_file.read('project.ini')

['project.ini']

In [ ]:
IMAGES_FOLDER_PATH = pathlib.Path(config_file.get('default', 'images_folder'))
# TODO: IMAGES = IMAGES_FOLDER_PATH.glob('**/*')
IMAGES_FOLDER_PATH

WindowsPath('C:/Users/Pende/Downloads/fashion')

## Django setup

Initialize the Django backend

In [4]:
import django
import os
from django.conf import settings

In [5]:
BASE_DIR = pathlib.Path('.')

In [6]:
settings.configure(
    BASE_DIR=BASE_DIR,
    INSTALLED_APPS=[
        'django.contrib.admin',
        'django.contrib.auth',
        'django.contrib.contenttypes',
        'django.contrib.sessions',
        'django.contrib.messages',
        'django.contrib.staticfiles',

        'django_extensions',
        'corsheaders',
        'rest_framework',
        'rest_framework.authtoken',
        
        'shop',
        'variants',
        'cart',
        'orders',
        'shipments',
        'reviews',
    ],
    DATABASES={
        'default': {
            'ENGINE': 'django.db.backends.postgresql',
            'NAME': 'mycommerce',
            'USER': 'test_user',
            'PASSWORD': 'touparet',
            'HOST': 'localhost',
            'PORT': '5432'
        }
    },
    MEDIA_ROOT=BASE_DIR.joinpath('media'),
    DEFAULT_AUTO_FIELD='django.db.models.BigAutoField',
    PRODUCT_MODEL='shop.Product',
    CUSTOMER_ORDERS_MODEL='orders.CustomerOrder',
    STRIPE_TOKENS=[()]
)


In [7]:
os.environ.setdefault('DJANGO_ALLOW_ASYNC_UNSAFE', 'True')

'True'

In [8]:
django.setup()

## File collection

Collect all the named images and videos in the images folder. The image's name will be used for the product's name. In that sense, similar images should have the same name in order for them to be considered the same product.

In [31]:
def collect_files():
    if os.path.exists(IMAGES_FOLDER_PATH):
        if os.path.isdir(IMAGES_FOLDER_PATH):
            result = list(os.walk(IMAGES_FOLDER_PATH))
            full_path, dirs, files = result[0]
            
            images = filter(lambda x: any([x.endswith('jpg'), x.endswith('jpeg')]), files)
            videos = filter(lambda x: x.endswith('mp4'), files)
            
            images = list(images)
            videos = list(videos)
            print('Collected', len(images), 'images and', len(videos), 'videos')
            
            return pathlib.Path(full_path), images, videos
    print('Collected', 0, 'images and', 0, 'videos')
    return None, [], []


# TODO: Use Pathlib
# def collect_files():
#     images = []
#     videos = []

#     for file in IMAGES:
#         if not file.is_file():
#             continue

#         if file.name.endswith('jpeg'):
#             images.append(file)
        
#         if file.name.endswith('mp4'):
#             videos.append(file)

#     print('Collected', len(images), 'images and', len(videos), 'videos')
#     return images, videos

## Product creation and fixtures

Creates the related product for each image in the database backend. Creates a JSON fixture file for testing the proeject.

In [32]:
import json
import pathlib
import re
import random
import datetime
import pytz
from shop.utils import create_product_slug
from shop.choices import ColorChoices, CategoryChoices
from django.utils.crypto import get_random_string


In [33]:
def detect_media_name(name):
    """Detects the true name of the media file"""
    result = re.match(r'(\w.*)(?=\(\d+\))', name)
    if result is not None:
        return result.group(0).strip().lower()
    else:
        return get_random_string(10)

In [34]:
def create_product_fixtures(model_dotted_path=None, create_fixture=False):
    with open(BASE_DIR / 'fixtures.json', mode='w', encoding='utf-8') as f:
        products = []

        _, images, _ = collect_files()
        names = set(map(detect_media_name, images))

        timzone = pytz.timezone('America/Chicago')
        current_date = str(datetime.datetime.now(tz=timzone).date())
        
        for i, name in enumerate(names):
            if name is not None:
                color = str(random.choice(ColorChoices.choices)[0])
                
                product = {
                    'name': name.title(),
                    'color': color,
                    'unit_price': random.randrange(10, 800),
                    'active': False,
                    'display_new': random.choice([True, False]),
                    'slug': create_product_slug(name, color),
                    'category': str(random.choice(CategoryChoices.choices)[0]),
                    'created_on': current_date,
                    'modified_on': current_date
                }

                is_on_sale = random.choice([True, False])
                product['on_sale'] = is_on_sale
                if is_on_sale:
                    product['sale_value'] = random.randrange(10, 50)
                else:
                    product['sale_value'] = 0
                
                if create_fixture:
                    if model_dotted_path is None:
                        raise ValueError('Fixtures require a model path e.g. app.MyModel')
                    products.append({
                        'pk': i + 1,
                        'model': model_dotted_path,
                        'fields': product
                    })

                    json.dump(products, f)
                    print('Created', len(names), 'products')
                else:
                    products.append(product)
        return products

# create_product_fixtures('shop.Product')


In [35]:
PRODUCTS = create_product_fixtures()

Collected 298 images and 36 videos


## Database object creation

Save all of the items in the database

In [36]:
from shop.models import Image, Video, Product
from shop.utils import process_file_name
from django.contrib.auth import get_user_model
from django.core.files import File


In [37]:
def create_media_objects(commit=False, images_alone=False):
    full_path, images, videos = collect_files()
    
    def image_iterator():
        for i, image in enumerate(images):
            f = open(full_path.joinpath(image), mode='rb')
            base_name, _, _ = process_file_name(image)
            clean_name = detect_media_name(base_name)
            yield Image(name=f"{clean_name}{i}", original=File(f, name=image))
            
    def video_iterator():
        for i, video in enumerate(videos):
            f = open(os.path.join(full_path, video), mode='rb')
            base_name, _, _ = process_file_name(video)
            clean_name = detect_media_name(base_name)
            yield Video(name=f"{clean_name}{i}", content=File(f, name=video))

    if commit:
        Image.objects.bulk_create(image_iterator())
        
        if not images_alone:
            Video.objects.bulk_create(video_iterator())
    else:
        return image_iterator(), video_iterator()

In [38]:
def create_products(commit=False):
    def iterator():
        for data in PRODUCTS:
            yield Product(**data)

    if commit:
        Product.objects.bulk_create(iterator())

In [39]:
create_media_objects(commit=True, images_alone=False)

Collected 298 images and 36 videos


In [18]:
create_products(commit=True)

In [19]:
USER_MODEL = get_user_model()

def create_users():
    users = [
        ['lucile@gmail.com', 'Lucile', 'Lopez', 'touparette'],
        ['pauline@gmail.com', 'Pauline', 'Matthieu', 'touparette']
    ]
    for user in users:
        try:
            user_object = USER_MODEL.objects.create_user(email=user[0], password=user[-1], first_name=user[1], last_name=user[2])
        except:
            print('User exists already')